In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import HTML, Math
import pandas as pd
from matplotlib import pyplot as plt

def cov_to_inf(X, domain):
    """
    Converts covariance form to influence diagram form

    Parameters:
    X (numpy.ndarray): Gaussian distribution (covariance matrix)
    domain (int): the number of rows and columns of X (assuming X is square)

    Returns:
    B (numpy.ndarray): An n x n matrix of Gaussian influence diagram arc coefficients which is strictly upper triangular
    V (numpy.ndarray): An n x 1 matrix of Gaussian influence diagram conditional variances with non-negative entries (including inf)
    P (numpy.ndarray): The (Moore-Penrose generalized) inverse of X
    """
    
    P = np.zeros((domain, domain))
    V = np.zeros(domain)
    B = np.zeros((domain, domain))

    # Initialize first row
    B[0, 0] = 0
    V[0] = X[0, 0]

    if V[0] == 0 or V[0] == np.inf:
        P[0, 0] = 0
    else:
        P[0, 0] = 1 / V[0]

    # Iterate through the domain
    for j in range(1, domain):
        B[j, j] = 0
        B[j, :j] = 0
        B[:j, j] = P[:j, :j].dot(X[:j, j])

        if X[j, j] == np.inf:
            V[j] = np.inf
        else:
            V[j] = X[j, j] - np.dot(X[j, :j], B[:j, j])
            V[j] = max(V[j], 0)

        if V[j] == 0 or V[j] == np.inf:
            P[j, j] = 0
            P[j, :j] = 0
            P[:j, j] = 0
        else:
            P[j, j] = 1 / V[j]
            for k in range(j):
                temp = P[j, j] * B[k, j]
                if k != 0:
                    P[k, :k] += temp * B[:k, j]
                    P[:k, k] = P[k, :k]
                P[k, k] += temp * B[k, j]
            P[j, :j] = -P[j, j] * B[:j, j]
            P[:j, j] = P[j, :j]

    return B, V.reshape(-1, 1), P

def evidence(u, B, V, X1, n0, n1, n2, du):
    """
    Enter evidence in influence diagram.

    Parameters:
    u (numpy.ndarray): A matrix with values including the mean of the state X(k) at discrete time k and the product of this mean and measurement matrix at discrete time k that maps the state X(k).
    B (numpy.ndarray): Matrix composed of covariance matrix of state X(k) and the measurement matrix at discrete time k.
    V (numpy.ndarray): A vector combining conditional variances with entries that are non-negative (including inf) and the measurement noise values.
    X1 (numpy.ndarray): Vector of n1 values with evidence in multivariate Gaussian with Influence Diagram form.
    n0 (int): Size of X0, where X0 is the predecessor of X1.
    n1 (int): Size of X1.
    n2 (int): Size of X2, where X2 is the successor of X1.
    du (numpy.ndarray): The change in u caused by the observed variable.

    Returns:
    u (numpy.ndarray): The updated mean vector of the state X(k) changed by the effect of the observed variable.
    B (numpy.ndarray): The updated matrix with strictly upper triangular submatrices and observed values set to 0.
    V (numpy.ndarray): The updated vector with non-negative entries (including inf) with the observed value set to 0.
    """
    
    for j in range(n1):
        B, V = reversal(B, V, 0, n0 + j, 1, n1 - j + n2)
        
        du[n0 + j] = X1[j] - u[n0 + j]
        du[:n0] = du[n0 + j] * B[n0 + j, :n0]
        
        if n0 + n1 + n2 >= n0 + j + 1:
            du[n0 + j + 1:n0 + n1 + n2] = du[n0 + j] * B[n0 + j, n0 + j + 1:n0 + n1 + n2]

        if n0 >= 2:
            for k in range(1, n0):  # Adjusting for 0-indexing in Python
                du[k] += np.dot(B[:k, k], du[:k])

        u[:n0] += du[:n0]

        if n1 + n2 >= j + 1:
            for k in range(n0 + j + 1, n0 + n1 + n2):
                du[k] += np.dot(B[:n0, k], du[:n0])
                du[k] += np.dot(B[n0 + j + 1:n0 + n1 + n2, k], du[n0 + j + 1:n0 + n1 + n2])
                u[k] += du[k]

        u[n0 + j] = 0
        V[n0 + j] = 0
        B[n0 + j, :n0 + n1 + n2] = 0

    return u, B, V

class trackingKF:
    def __init__(self, F, H, state, state_covariance, process_noise):
        self.StateTransitionModel = F
        self.MeasurementModel = H
        self.State = state
        self.StateCovariance = state_covariance
        self.ProcessNoise = process_noise

# Define the IDcorrect function as previously provided
def id_correct(filter, zmeas, zcov):
    u = filter.State
    P = filter.StateCovariance
    H = filter.MeasurementModel
    F = filter.StateTransitionModel
    Qk = filter.ProcessNoise
    M = Qk.shape
    Vzeros = np.zeros((M[0], 1))
    I = np.eye(M[0])

    # Perform measurement update
    u, V, B = mupdate(0, zmeas, u, P, Vzeros, zcov, H)
    '''print("Corrected state (u):", u)
    print("V:", V)
    print("B:", B)'''

    # Get the shape of B and set up L
    col_L, row_L = B.shape
    L = col_L
    B = inf_to_cov(V, B, L)

    # Set corrected values
    xcorr = u
    Pcorr = B

    return xcorr, Pcorr

def id_predict(filter):
    """
    Performs the Time Update (prediction) portion of the Kalman Filter for object-oriented programming.

    Parameters:
    filter (object): A trackingKF object that contains State, StateCovariance, MeasurementModel, StateTransitionModel, and ProcessNoise.

    Returns:
    xpred (numpy.ndarray): The predicted state.
    Ppred (numpy.ndarray): The predicted state estimation error covariance.
    """

    # Extract filter parameters
    u = filter.State
    B_old = filter.StateCovariance
    H = filter.MeasurementModel
    F = filter.StateTransitionModel
    Qk = filter.ProcessNoise
    
    M = Qk.shape[0]
    I = np.eye(M)

    # Convert covariance to influence diagram form
    B_old, V_old, Precision = cov_to_inf(B_old, M)

    # Perform time update
    u, B, V = tupdate(u, B_old, V_old, F, I, Qk)

    # Convert influence diagram back to covariance form
    Ppred = inf_to_cov(V, B, M)

    xpred = u

    return xpred, Ppred

def inf_to_cov(V, B, domain):
    """
    Converts influence diagram form to covariance form.

    Parameters:
    V (numpy.ndarray): An n x 1 vector with non-negative (including inf) entries.
    B (numpy.ndarray): An n x n matrix that is strictly upper triangular.
    domain (int): The number of rows and columns of B.

    Returns:
    X (numpy.ndarray): The covariance matrix of the multivariate Gaussian distribution.
    """

    # Initialize the covariance matrix X
    X = np.zeros((domain, domain))
    
    # First element in the diagonal
    X[0, 0] = V[0]

    for i in range(1, domain):
        for j in range(i):
            X[i, j] = 0
            for k in range(i):
                if X[j, k] != np.inf:
                    X[i, j] += X[j, k] * B[k, i]
            X[j, i] = X[i, j]  # Since the matrix is symmetric

        # Update diagonal elements
        if V[i] == np.inf:
            X[i, i] = np.inf
        else:
            Y = X[i, :i]
            Z = B[:i, i]
            X[i, i] = V[i] + np.dot(Y, Z)

    return X

def kalman(k, Z, u, X, V, R, H, Phi, gamma, Qk, Form, h=None):
    """
    Apply Kalman filter at time k.

    Parameters:
    k (int): Desired discrete time.
    Z (numpy.ndarray): Measurement values at discrete time k.
    u (numpy.ndarray): An n x 1 vector representing the mean of the state at time k.
    X (numpy.ndarray): If k=0, the covariance matrix of state at time k. If k≠0, the matrix of Gaussian influence diagram arc coefficients.
    V (numpy.ndarray): If k≠0, an n x 1 vector of Gaussian influence diagram conditional variances. Ignored if k=0.
    R (numpy.ndarray): The measurement noise covariance matrix R.
    H (numpy.ndarray): The measurement matrix at discrete time k.
    Phi (numpy.ndarray): The state transition matrix at time k.
    gamma (numpy.ndarray): The process noise matrix at time k.
    Qk (numpy.ndarray): The process noise covariance matrix.
    Form (int): Determines the output form (0 for ID form, 1 for covariance form).

    Returns:
    u (numpy.ndarray): The updated mean vector.
    B (numpy.ndarray): The updated state covariance matrix or influence diagram form matrix.
    V (numpy.ndarray): The updated vector of conditional variances.
    """

    # Get dimensions
    domain = X.shape[0]
    p = Z.shape[0]
    
    # Perform measurement update
    u, V, B = mupdate(k, Z, u, X, V, R, H, h)
    u_new = u[:domain]
    V_new = V[:domain]
    B_new = B[:domain, :domain]

    # Perform time update
    u, B, V = tupdate(u_new, B_new, V_new, Phi, gamma, Qk)

    # Convert back to covariance form if required
    if Form == 1:
        B = inf_to_cov(V, B, domain)

    return u, B, V

def mupdate(k, Z, u, B_or_sigma, V, R, H, h=None):
    """
    Mupdate
    Measurement update for measurement Z(k)
    
    Inputs:
    k - desired discrete time
    Z - p x 1 vector of measurement values at discrete time k
    u - n x 1 vector that represents the mean of the state X(k) at discrete time k
    B_or_sigma - If k == 0, n x n covariance matrix of state X(k) at discrete time k.
                 If k != 0, n x n matrix of Gaussian influence diagram arc coefficients of state X(k).
    V - If k != 0, n x 1 vector of Gaussian influence diagram conditional variances.
        If k == 0, this input is ignored.
    R - p x p diagonal measurement noise covariance matrix R
    H - p x n measurement matrix at discrete time k
    
    Outputs:
    u - updated n x 1 vector representing the mean of the state X(k+1)
    V - updated n x 1 vector of Gaussian influence diagram conditional variances of state X(k+1)
    B - updated n x n matrix of Gaussian influence diagram arc coefficients of state X(k+1)
    """

    # Determine dimensions of V and Z
    domain = V.shape[0]
    n = domain
    p = Z.shape[0]

    if k == 0:
        B, V, P = cov_to_inf(B_or_sigma, domain)

    # Prepare intermediate values for update
    X1 = np.array(Z).reshape(-1, 1)
    n0 = n
    n1 = p
    n2 = 0
    u_new = np.vstack((u, H @ u))
    if h is not None:
        u_new = np.vstack((u, h))

    #du = np.zeros((2, 1))
    '''print('V\n', V)
    print('R\n', R)'''

    # Construct V_new to match MATLAB structure
    V_new = np.vstack((V, np.diag(R).reshape(-1, 1)))
    #print('V_new\n', V_new)

    # Build the B_new matrix
    Opn = np.zeros((p, n))
    Opp = np.zeros((p, p))
    B_new = np.block([[B, H.T], [Opn, Opp]])

    V_new = V_new.T
    V_new = V_new.flatten()
    u_new = u_new.T
    u_new = u_new.flatten()
    X1 = X1.T
    X1 = X1.flatten()
    du = u_new.copy()

    '''print('V_new\n', V_new)
    print('B_new\n', B_new)
    print('u_new\n', u_new)
    print('X1\n', X1)
    print('n0\n', n0)
    print('n1\n', n1)
    print('n2\n', n2)
    print('du\n', du)'''

    # Update using Evidence function
    u, B, V = evidence(u_new, B_new, V_new, X1, n0, n1, n2, du)

    # Return only the relevant portions
    u = u[:n]
    B = B[:n, :n]
    V = V[:n]

    return u, V, B

def removal(B, V, n0, n1, n2):
    """
    Removal of vector nodes in Gaussian influence diagram.

    Parameters:
    B (numpy.ndarray): An n x n strictly upper triangular matrix comprised of strictly upper triangular submatrices.
    V (numpy.ndarray): An n x 1 vector with non-negative (including inf) entries.
    n0 (int): The size of vector node x0.
    n1 (int): The size of vector node x1.
    n2 (int): The size of vector node x2.

    Returns:
    B (numpy.ndarray): Updated n x n matrix with removed vector nodes.
    V (numpy.ndarray): Updated n x 1 vector with removed vector nodes.
    """

    # If n2 > 1, reverse arcs from vector node x1 to the first n2-1 elements of vector node x2
    if n2 > 1:
        B, V = reversal(B, V, n0, n1, n2-1, 0)
        '''print(B)
        print(V)'''

    N = n0 + n1 + n2

    # Iteratively remove elements of the vector nodes
    for i in range(n0 + n1, n0, -1):
        if n0 >= 1:
            B[:n0, N - 1] += B[i - 1, N - 1] * B[:n0, i - 1]

        if i - 1 > n0:
            B[n0:i-1, N - 1] += B[i - 1, N - 1] * B[n0:i-1, i - 1]

        if N - 1 > n0 + n1:
            B[n0 + n1:N - 1, N - 1] += B[i - 1, N - 1] * B[n0 + n1:N - 1, i - 1]

        if V[i - 1] != 0:
            # Check for infinity before updating V[N-1]
            if V[i - 1] != np.inf and V[N - 1] != np.inf:
                V[N - 1] += B[i - 1, N - 1] * B[i - 1, N - 1] * V[i - 1]
            else:
                V[N - 1] = np.inf

    # Set appropriate entries in V and B to 0 after removing node x1
    V[n0:n0 + n1] = 0
    B[n0:n0 + n1, :] = 0
    B[:, n0:n0 + n1] = 0

    return B, V

def reversal(B, V, n0, n1, n2, n3):
    """
    Arc reversal between two nodes using Bayes' rule.

    Parameters:
    B (numpy.ndarray): An n x n strictly upper triangular matrix, composed of strictly upper triangular submatrices.
    V (numpy.ndarray): An n x 1 vector with non-negative (including inf) entries.
    n0 (int): Size of vector node x0.
    n1 (int): Size of vector node x1.
    n2 (int): Size of vector node x2.
    n3 (int): Size of vector node x3.

    Returns:
    B (numpy.ndarray): Updated matrix with reversed arcs.
    V (numpy.ndarray): Updated vector with adjusted variances.
    """

    # Iterate from n0 + n1 down to n0 + 1
    for i in range(n0 + n1, n0, -1):
        for j in range(n0 + n1 + 1, n0 + n1 + n2 + 1):
            if B[i - 1, j - 1] != 0:
                # Update the matrix B by adjusting the appropriate elements
                if n0 >= 1:
                    B[:n0, j - 1] += B[i - 1, j - 1] * B[:n0, i - 1]
                
                if i - 1 > n0:
                    B[n0:i-1, j - 1] += B[i - 1, j - 1] * B[n0:i-1, i - 1]

                if j - 1 > n0 + n1:
                    B[n0 + n1:j - 1, j - 1] += B[i - 1, j - 1] * B[n0 + n1:j - 1, i - 1]
                
                # Update based on the variance matrix V
                if V[i - 1] == 0:
                    B[j - 1, i - 1] = 0
                else:
                    if V[i - 1] != np.inf and V[j - 1] != np.inf:
                        if V[j - 1] == 0:
                            V[j - 1] = B[i - 1, j - 1] ** 2 * V[i - 1]
                            V[i - 1] = 0
                            B[j - 1, i - 1] = 1 / B[i - 1, j - 1]
                        else:
                            Vj_old = V[j - 1]
                            V[j - 1] += B[i - 1, j - 1] ** 2 * V[i - 1]
                            V_ratio = V[i - 1] / V[j - 1]
                            V[i - 1] = Vj_old * V_ratio
                            B[j - 1, i - 1] = B[i - 1, j - 1] * V_ratio
                    else:
                        if V[j - 1] != np.inf:
                            B[j - 1, i - 1] = 1 / B[i - 1, j - 1]
                        else:
                            B[j - 1, i - 1] = 0
                        
                        if V[i - 1] == np.inf and V[j - 1] != np.inf:
                            V[i - 1] = V[j - 1] * B[j - 1, i - 1] ** 2
                        
                        V[j - 1] = np.inf

                # Zero out the current entry
                B[i - 1, j - 1] = 0

                # Further update B based on the reversal process
                if n0 >= 1:
                    B[:n0, i - 1] -= B[j - 1, i - 1] * B[:n0, j - 1]
                
                if i - 1 > n0:
                    B[n0:i-1, i - 1] -= B[j - 1, i - 1] * B[n0:i-1, j - 1]
                
                if j - 1 > n0 + n1:
                    B[n0 + n1:j - 1, i - 1] -= B[j - 1, i - 1] * B[n0 + n1:j - 1, j - 1]

    return B, V

def tupdate(u, B, V, Phi, gamma, Qk):
    """
    Time update from X(k) to X(k+1) in the Kalman filter.

    Parameters:
    u (numpy.ndarray): An n x 1 vector representing the mean of the state X(k) at time k.
    B (numpy.ndarray): An n x n matrix of Gaussian influence diagram arc coefficients of state X(k) at time k.
    V (numpy.ndarray): An n x 1 vector of Gaussian influence diagram conditional variances of state X(k).
    Phi (numpy.ndarray): The n x n state transition matrix Phi(k).
    gamma (numpy.ndarray): The n x r process noise matrix Gamma(k).
    Qk (numpy.ndarray): The r x r process noise covariance matrix of the process noise vector w(k).

    Returns:
    u (numpy.ndarray): The updated mean vector of the state X(k+1) at time k+1.
    B (numpy.ndarray): The updated matrix of Gaussian influence diagram arc coefficients of state X(k+1).
    V (numpy.ndarray): The updated vector of conditional variances of state X(k+1).
    """

    # Get dimensions
    n = u.shape[0]  # size of u
    r = Qk.shape[0]  # size of Qk

    # Convert process noise covariance matrix to influence diagram form
    Bq, Vq, _ = cov_to_inf(Qk, r)
    #this part works

    # Prepare block matrices
    On = np.zeros((n, n))
    Or = np.zeros((r, r))
    Onr = np.zeros((n, r))
    On1 = np.zeros((n, 1))
    Or1 = np.zeros((r, 1))

    # Create new V and B matrices for the update
    V_new = np.concatenate((Vq.reshape(-1, 1), V.reshape(-1, 1), On1), axis=0)
    B_new = np.block([
        [Bq, Onr.T, gamma.T],
        [Onr, B, Phi.T],
        [Onr, On, On]
    ])
    #this works


    
    # Perform Removal operation to update B and V
    n0 = 0
    n1 = r
    n2 = n+n
    #n3 = 0

    '''print('mibomba')
    print(B_new)
    print(V_new)
    print(n0)
    print(n1)
    print(n2)'''

    #this part works
    B_temp, V_temp = removal(B_new, V_new, n0, n1, n2)

    '''print('mibomba')
    print(V_temp)
    print(B_temp)'''

    n0 = r
    n1 = n
    n2 = n
    '''print('hibomba')
    print(r)
    print(n)'''

    V_temp = V_temp.flatten()
    B, V = removal(B_temp, V_temp, n0, n1, n2)
    '''print('dibomba')
    print(B)
    print(V)'''


    # Update V and B for the current step
    V = V.flatten()  # Ensure V is a flat vector
    B = B[n+r:n+r+n, n+r:n+r+n]  # Extract the relevant part of B
    V = V[n+r:n+r+n]  # Extract the relevant part of V

    # Update the mean vector
    u_new = Phi @ u  # Update u using the state transition matrix
    u = u_new

    return u, B, V


# Function to create a grid of FloatText widgets with default values and dynamic dimensions
def create_matrix_input(description, default_values, rows, cols):
    matrix_label = widgets.Label(value=description)
    rows_input = widgets.IntText(
        value=rows, 
        description="Rows:", 
        layout=widgets.Layout(width="150px", height="40px")
    )
    cols_input = widgets.IntText(
        value=cols, 
        description="Cols:", 
        layout=widgets.Layout(width="150px", height="40px")
    )
    update_button = widgets.Button(description="Update Dimensions", button_style="info")
    matrix_grid = widgets.GridBox(
        children=[
            widgets.FloatText(
                value=default_values[i, j] if i < default_values.shape[0] and j < default_values.shape[1] else 0,
                layout=widgets.Layout(width="80px")
            )
            for i in range(rows) for j in range(cols)
        ],
        layout=widgets.Layout(grid_template_columns=f"repeat({cols}, 100px)")
    )

    # Function to update the grid based on new dimensions
    def update_grid(_):
        nonlocal matrix_grid
        new_rows = max(1, rows_input.value)  # Ensure at least 1 row
        new_cols = max(1, cols_input.value)  # Ensure at least 1 column
        matrix_grid.children = [
            widgets.FloatText(value=0, layout=widgets.Layout(width="80px"))
            for _ in range(new_rows * new_cols)
        ]
        matrix_grid.layout = widgets.Layout(grid_template_columns=f"repeat({new_cols}, 100px)")
        with output:
            clear_output()
            print(f"Updated {description} to {new_rows}x{new_cols} dimensions.")
        display(matrix_grid)

    update_button.on_click(update_grid)

    return widgets.VBox([matrix_label, rows_input, cols_input, update_button, matrix_grid]), rows_input, cols_input, matrix_grid

# Function to parse the matrix grid into a NumPy array
def parse_matrix(grid, rows, cols, description):
    try:
        values = [float(child.value) for child in grid.children]
        return np.array(values).reshape(rows, cols)
    except Exception as e:
        raise ValueError(f"Error parsing {description}: {e}")

# Default values
Z_default = np.array([[502.55], [-0.9316]])
u_default = np.array([[400], [0], [0], [-300], [0], [0]])
X_default = np.array([
    [1125, 750, 250, 0, 0, 0],
    [750, 1000, 500, 0, 0, 0],
    [250, 500, 500, 0, 0, 0],
    [0, 0, 0, 1125, 750, 250],
    [0, 0, 0, 750, 1000, 500],
    [0, 0, 0, 250, 500, 500]
])
V_default = np.array([[0], [0], [0], [0], [0], [0]])
R_default = np.array([[25, 0], [0, 0.0087**2]])
H_default = np.array([
    [0.8, 0, 0, -0.6, 0, 0],
    [0.0012, 0, 0, 0.0016, 0, 0]
])
Phi_default = np.array([
    [1, 1, 0.5, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 0.5],
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 1]
])
gamma_default = np.eye(6)
Qk_default = np.array([
    [0.25, 0.5, 0.5, 0, 0, 0],
    [0.5, 1, 1, 0, 0, 0],
    [0.5, 1, 1, 0, 0, 0],
    [0, 0, 0, 0.25, 0.5, 0.5],
    [0, 0, 0, 0.5, 1, 1],
    [0, 0, 0, 0.5, 1, 1]
]) * 0.2**2
h_default = np.array([[500], [-0.644]])

# Preloaded UI with dynamic dimensions
Z_input, Z_rows, Z_cols, Z_grid = create_matrix_input("Z (Measurement Values):", Z_default, 2, 1)
u_input, u_rows, u_cols, u_grid = create_matrix_input("u (State Mean Vector):", u_default, 6, 1)
X_input, X_rows, X_cols, X_grid = create_matrix_input("X (Covariance Matrix):", X_default, 6, 6)
V_input, V_rows, V_cols, V_grid = create_matrix_input("V (Conditional Variances):", V_default, 6, 1)
R_input, R_rows, R_cols, R_grid = create_matrix_input("R (Measurement Noise Covariance):", R_default, 2, 2)
H_input, H_rows, H_cols, H_grid = create_matrix_input("H (Measurement Matrix):", H_default, 2, 6)
Phi_input, Phi_rows, Phi_cols, Phi_grid = create_matrix_input("Phi (State Transition Matrix):", Phi_default, 6, 6)
gamma_input, gamma_rows, gamma_cols, gamma_grid = create_matrix_input("gamma (Process Noise Matrix):", gamma_default, 6, 6)
Qk_input, Qk_rows, Qk_cols, Qk_grid = create_matrix_input("Qk (Process Noise Covariance):", Qk_default, 6, 6)
h_input, h_rows, h_cols, h_grid = create_matrix_input("h (Expected Measurement Values):", h_default, 2, 1)
Form_input = widgets.Dropdown(
    options=["ID Form", "Covariance Form"],
    value="Covariance Form",
    description="Output Form:"
)
run_button = widgets.Button(description="Run Kalman Filter", button_style="success")

# Output display
output = widgets.Output()

# Run Kalman filter
def run_kalman_filter(_):
    with output:
        clear_output()
        try:
            k = 0  # Fixed discrete time
            Z = parse_matrix(Z_grid, Z_rows.value, Z_cols.value, "Measurement values (Z)")
            u = parse_matrix(u_grid, u_rows.value, u_cols.value, "State mean vector (u)")
            X = parse_matrix(X_grid, X_rows.value, X_cols.value, "Covariance matrix (X)")
            V = parse_matrix(V_grid, V_rows.value, V_cols.value, "Conditional variances (V)")
            R = parse_matrix(R_grid, R_rows.value, R_cols.value, "Measurement noise covariance (R)")
            H = parse_matrix(H_grid, H_rows.value, H_cols.value, "Measurement matrix (H)")
            Phi = parse_matrix(Phi_grid, Phi_rows.value, Phi_cols.value, "State transition matrix (Phi)")
            gamma = parse_matrix(gamma_grid, gamma_rows.value, gamma_cols.value, "Process noise matrix (gamma)")
            Qk = parse_matrix(Qk_grid, Qk_rows.value, Qk_cols.value, "Process noise covariance (Qk)")
            h = parse_matrix(h_grid, h_rows.value, h_cols.value, "Expected measurement values (h)")
            Form = 0 if Form_input.value == "ID Form" else 1
            
            # Run Kalman filter
            u_updated, B_updated, V_updated = kalman(k, Z, u, X, V, R, H, Phi, gamma, Qk, Form, h)
            
            # Display results
            def display_matrix(matrix, title):
                """
                Print a matrix in plain text format for readability.

                Parameters:
                matrix (numpy.ndarray): The matrix to display.
                title (str): The title for the matrix.
                """
                print(f"{title}:")
                print(matrix)
                print()  # Add a blank line for spacing

            display_matrix(u_updated, "Updated State (u)")
            display_matrix(B_updated, "Updated Covariance Matrix (B or X)")
            display_matrix(V_updated, "Updated Conditional Variances (V)")


        
        except Exception as e:
            print(f"Error: {e}")

# Attach the event handler
run_button.on_click(run_kalman_filter)

# Display the input UI
display(
    widgets.VBox([
        Z_input, u_input, X_input, V_input, R_input, 
        H_input, Phi_input, gamma_input, Qk_input, 
        Form_input, h_input, run_button, output
    ])
)